In [1]:
using LinearAlgebra
using Plots
using Printf
using SparseArrays
using Arpack
using Statistics
using LaTeXStrings
using BenchmarkTools
using Octavian
using StaticArrays
using Profile
using Random
using Distributions;

In [18]:
function func_DQMC(hopping_matrix,D0,U0,g,M,Omega,U,beta,N_x,N_y,dT,dk,n_markov,N_sw_warm_up,N_sw_measurement,measurement_time_step,swtch_correlation)
    
    N_s = N_x * N_y; #Number of sites
    
    N_T = Int(max(dk,round(beta/dT)));#number of time steps
    dT = beta/N_T; #time step
    
    #parameters
    lambda = acosh(exp(U*dT/2));
    M = M;
    g = g;
    Omega = Omega;
    
    # to be used for Green's function evaluation and space-time wrap
    k_vec = zeros(1,N_T)::Matrix{Float64};
    k_vec[dk:dk:end] .= 1.0;
    
    N_sweeps = N_sw_warm_up + N_sw_measurement;
    
    ##############################################
    #  computing expm(KdT), expm...........
    ###############################################
    
    B = U0*Diagonal(exp.(-dT*D0))*U0'; #--> Kinetic term in H_el
    B_root = U0*Diagonal(exp.(-dT*D0/2))*U0';
    inv_B = U0*Diagonal(exp.(dT*D0))*U0';
    inv_B_root = U0*Diagonal(exp.(dT*D0/2))*U0';
    
    #######################################
    num_samples_avg = zeros(n_markov,1)::Matrix{Float64};
    Sgn_avg = Dict();
    GF_up_avg = Dict();
    GF_dn_avg = Dict();
    n_up_dn_avg = Dict();
    abs_log = Vector{Float64}();
  
    for count = 1:n_markov

        progress = [count count/n_markov*100]
        @printf("Progress = %s\n", (Int.(progress)))
        
        # Initalization:
        num_samples = 0;
        Sgn_Z = 0.0;
        Sgn_Z::Float64;
        sgn_Z_accumulated = 0.0;
        sgn_Z_accumulated::Float64;
        
        n_up_dn = zeros(N_s)::Vector{Float64};
        GF_up = zeros(N_s,N_s)::Matrix{Float64};
        GF_dn = zeros(N_s,N_s)::Matrix{Float64};
        
        # Initialization with a random seed for Hubbard-Stratonovic fields:
        s = rand([-1,1],(N_s,N_T)); #Initializing Hubbard-Stratonivc fields witha random binary values
        h = lambda*s; # h_up = lambda*s    ,    h_dn = -h_up = -lambda*s
        X = rand(Uniform(-1,1),(N_s,N_T)); #random configuration for phonon fields #rows = sites, columns = times
        
        
        # computing corresponding Green's functions for up and down spin electrons
        B_up = Cluster(B,inv_B,h,X,N_T,g,dT);  # computing B_up(l) = exp(-KdT)exp(h_up(l)) for all l = 1:N_T      h_up =  h =  lambda*s
        B_dn = Cluster(B,inv_B,-h,X,N_T,g,dT); # computing B_dn(l) = exp(-KdT)exp(h_dn(l)) for all l = 1:N_T      h_dn = -h = -lambda*s
        
        # Using QR decomposition to stabilize B_L ... B_1 to compute G = inv(Id + B_L ... B_1)  for B = B_up or B_dn
        G_up,G_dn,log_det_G,sign_det_G = Initial_Evaluation(B_up,B_dn,k_vec,N_T);
        
        # initial sign of det(Z) = 1/det(G_up*G_dn)
        # the sign of various HS and Phonon fields/configs is needed for computing expectation values
        sgn_Z_accumulated = sign_det_G;

        # Space-time sweeps:
        for nsw = 1:(N_sw_warm_up + N_sw_measurement)
            
            # loop over various time-slices
            for l = 1:N_T
    
                ###############
                # local update
                ###############
                
                # Green's function update upon Hubbard-Stratonovic fields flip and Phonon fields update
                # using Sherman-Morrison update method
                G_up,G_dn,h,X,sgn_Z_accumulated_tmp = Sher_Mor(G_up,G_dn,h,X,N_s,N_T,M,Omega,dT,g); #time = l
    
                ### note that we need the sign of current GF with respect to the first GF so:sgn_current = sgn_prev*sgn_tmp
                sgn_Z_accumulated = sgn_Z_accumulated*sgn_Z_accumulated_tmp;
                
                #next time slice
                h = circshift(h,[0,-1]);                     
                X = circshift(X,[0,-1]);
                k_vec = circshift(k_vec,-1);
                
                if k_vec[l] == 1
                    
                    B_up = Cluster(B,inv_B,h,X,N_T,g,dT);
                    B_dn = Cluster(B,inv_B,-h,X,N_T,g,dT);
                    
                    G_up,G_dn = Initial_Evaluation(B_up,B_dn,k_vec,N_T);
                    
                else
                    #update is not accepted and we use wrap-up
                    ### spacetime wrapping using already existing Green's functions
                    h_tmp = h[:,N_T];
                    X_tmp = X[:,N_T];
                    
                    # spin_up:
                    B_up_prev = B*Diagonal(exp.(h_tmp - g*dT*X_tmp));
                    inv_B_up_prev = Diagonal(exp.(g*dT*X_tmp - h_tmp))*inv_B;

                    G_up = matmul(B_up_prev,matmul(G_up,inv_B_up_prev));
                    #G_up = B_up_prev*G_up*inv_B_up_prev;
                    
                    # spin dn:
                    B_dn_prev = B*Diagonal(exp.(-h_tmp - g*dT*X_tmp));
                    inv_B_dn_prev = Diagonal(exp.(g*dT*X_tmp + h_tmp))*inv_B;
                    
                    G_dn = matmul(B_dn_prev,matmul(G_dn,inv_B_dn_prev));
                    #G_dn = B_dn_prev*G_dn*inv_B_dn_prev;
                end

                ## checks whether or not measurements are in order in this time-step
                swtch_time_measurement = 0;
                if mod(l-1,measurement_time_step) == 0
                    swtch_time_measurement = 1;
                end
                if  nsw > N_sw_warm_up
                    if swtch_time_measurement == 1
                        num_samples = num_samples + 1;

                        Sgn_Z = Sgn_Z + sgn_Z_accumulated;
                        
                        ### inv_B_root*G_up*B_root & inv_B_root*G_dn*B_root
                        ### instead of G_up and G_dn are used to employ the
                        ### 2nd order Trotter-Suzuki decomposition (which is more accurate)               
                        
                        GF_up_tmp = I -  matmul(inv_B_root,matmul(G_up,B_root));
                        GF_dn_tmp = I -  matmul(inv_B_root,matmul(G_dn,B_root));

                        ### spin-dependent density profile for current Hubbard-Stratonbvic fields
                        n_up_tmp = diag(GF_up_tmp);
                        n_dn_tmp = diag(GF_dn_tmp);

                        ### spin sz and total density profile for current Hubbard-Stratonbvic fields          
                        sz_tmp  = (n_up_tmp - n_dn_tmp)/2;
                        rho_tmp = (n_up_tmp + n_dn_tmp)/2;
                        #### to compute average GF and <n_up n_dn>
                        
                        mul!(GF_up, GF_up_tmp, sgn_Z_accumulated, 1., 1.)
                        
                        mul!(GF_dn, GF_dn_tmp, sgn_Z_accumulated, 1., 1.)
                        
                        BLAS.axpy!(sgn_Z_accumulated,broadcast(*,n_up_tmp,n_dn_tmp),n_up_dn)     
                    end
                end
            end
            

            ###################
            #  Global update 
            ###################
            
            Ns_gl = 3;
            sites = sample(1:N_s,Ns_gl,replace=false);
            gl_update = rand(Uniform(-1,1),Ns_gl)*transpose(ones(N_T))        

            ###### update site_i at all time slices, simultaneously
            for i = (1:Ns_gl)
                X[[sites[i]],:] += gl_update[[i],:]; 
            end
            
            B_up = Cluster(B,inv_B,h,X,N_T,g,dT);
            B_dn = Cluster(B,inv_B,-h,X,N_T,g,dT);

            G_up,G_dn,log_det_G,sign_det_G = Initial_Evaluation(B_up,B_dn,k_vec,N_T);

            sgn_Z_accumulated = sign_det_G; #update sign_z
            log_det_G = copy(log_det_G);

            push!(abs_log, abs(log_det_G));

        end
        
        ###<O> =  <Osign(Z)>/<sign(Z)>
        GF_up_avg[count] = GF_up/Sgn_Z;
        GF_dn_avg[count] = GF_dn/Sgn_Z;
        n_up_dn_avg[count] = n_up_dn/Sgn_Z;

        num_samples_avg[count] = num_samples;
        Sgn_avg[count] = Sgn_Z/num_samples; ### average sign of fermion determinants Z, where Z = 1/(Det(G_up)Det(G_dn)) for a given Hubbard-Stratonvic field
    end
    
    ### expection values for each markov chain is stored in a cell structure
    ###  for final statistical average in a separate function
    correlations_tmp = Dict();
    correlations_tmp[1] = GF_up_avg;
    correlations_tmp[2] = GF_dn_avg;
    correlations_tmp[3] = n_up_dn_avg;
    #println(n_up_dn_avg)

    #### saving average spin and of number of samples related to partition
    #### function in another cell structure.
    par_func_tmp = Dict();
    par_func_tmp[1] = num_samples_avg;
    par_func_tmp[2] = Sgn_avg;
    
    return par_func_tmp,correlations_tmp,abs_log
end                 

func_DQMC (generic function with 1 method)

In [19]:
function Cluster(B_K,inv_B_K,h,X,N_T,g,dT)
    
    # Here we compute B_up(l) = exp(-KdT)exp(h_up(l)) & B_dn(l) = exp(-KdT)exp(h_dn(l)) where h_up = -h_dn = h
    
    B = [B_K*Diagonal(exp.(h[:,l] - g*dT*X[:,l])) for l=1:N_T]
    B::Vector{Matrix{Float64}}
    return B
end

Cluster (generic function with 1 method)

In [20]:
function Strat(B,k_vec,N_T)
    
    N_s = size(B[1])[1]
    Q = diagm(ones(N_s))::Matrix{Float64};
    D = Diagonal(ones(N_s))::Diagonal{Float64, Vector{Float64}};
    T =  diagm(ones(N_s))::Matrix{Float64};
    
    ii = 1;
    while ii <= N_T
        
        B0 = B[ii];
        ii = ii +  1;
        while ii <= N_T && k_vec[ii] < 1

            B0 = B[ii]*B0;
            ii = ii + 1;
        end

        C = (B0*Q)*D;
        Q_0,R = qr(C);
        Q = convert(Matrix{Float64}, Q_0);
        D0 = diag(R);
        inv_D = (Diagonal(1 ./D0));
        D = Diagonal(D0);
        T = (inv_D*R)*T;
    end

    D_diag = diag(D);
    D_b = max.(1, abs.(D_diag)) .* sign.(D_diag);
    D_s = min.(1, abs.(D_diag)); 
    
    inv_D_b = (Diagonal(1 ./D_b));
    D_S = (Diagonal(D_s));

    A1 = inv_D_b*(Q') + D_S*T;
    A2 = (inv_D_b);
    A3 = Q';
    
    G = (A1\A2) * A3;

    ## computing log(det(abs(G))) as well as sign(det(G)) accurately and with special care (again to avoid numerical instabilities)
    
    L1,U1 = lu(A1,Val(false));
    
    log_det_L1 = 0;
    sign_det_L1 = sign(det(L1));
    
    log_det_U1, sign_det_U1 = logabsdet(U1);
    
    
    log_det_A1 = log_det_L1 + log_det_U1;
    sign_det_A1 = sign_det_L1*sign_det_U1;
    
    log_det_A2 = -sum(log.(broadcast(abs,D_b)));
    sign_det_A2 = prod(sign.(D_b));

    log_det_A3 = 0;
    sign_det_A3 = sign.(det(A3));

    sign_det_G = sign_det_A1*sign_det_A2*sign_det_A3; #sign of det(G)
    log_det_G = -log_det_A1 + log_det_A2 + log_det_A3; #log of abs(det(G))
    
    G::Matrix{Float64};
    sign_det_G::Float64;
    
    return G,log_det_G,sign_det_G
    
end 

Strat (generic function with 1 method)

In [21]:
function Sher_Mor(G_up,G_dn,h,X,N_s,N_T,M,Omega,dT,g)
    
    sgn_accumulated_tmp = 1.0;
    h = transpose(h);
    X = transpose(X);
    g = g;

    for i1 = 1:N_s
        
        ### ratio of 1/det(G_up)
        alpha_up = exp.(-2*h[1,i1]) - 1;
        r_up = 1 + alpha_up*(1-G_up[i1,i1]);
        
        ### ratio of 1/det(G_dn)
        alpha_dn = exp.(+2*h[1,i1]) - 1;
        r_dn = 1 + alpha_dn*(1-G_dn[i1,i1]);
        
        #total ratio
        r = r_up*r_dn;
        
        # Hubbard-Stratonic field flip acceptance based on Metropolis-Hasting algorithm
        # as a consequence Green's function update based on the Sherman-Morrison update formula

        if rand(1)[1] <= abs(r)
            
            # spin up GF update:
            a_up = (I - G_up);
            b_up = G_up;
            
            mul!(G_up, -matmul(a_up[:,[i1]],b_up[[i1],:]), (alpha_up/r_up), 1., 1.);
            
            #  spin dn GF update:
            a_dn = (I - G_dn);
            b_dn = G_dn;
            
            mul!(G_dn, -matmul(a_dn[:,[i1]],b_dn[[i1],:]), (alpha_dn/r_dn), 1., 1.);
            
            # update h of accumulated sign of det
            h[1,i1] = -h[1,i1];
            sgn_accumulated_tmp = sgn_accumulated_tmp * sign(r);
            sgn_accumulated_tmp::Float64;
            
            G_up::Matrix{Float64};
            G_dn::Matrix{Float64};
        
        end
        
        #phonon fields single site update
        delta_X = rand(Uniform(-1,1),1)[1];

        
        #before update:
        E_ph_prev = (M*Omega^2/2)*X[1,i1]^2 + (M/2 * dT^(-2))*((X[2,i1] - X[1,i1])^2 + (X[1,i1] - X[N_T,i1])^2);
        #after update:
        E_ph_after = (M*Omega^2/2)*(X[1,i1] + delta_X)^2 + (M/2 * dT^(-2))*((X[2,i1] - (X[1,i1] + delta_X))^2 + ((X[1,i1] + delta_X) - X[N_T,i1])^2);
        
        
        delta_E_ph = E_ph_after - E_ph_prev; #change in the phonon energy 
        beta = exp.(-dT*g*delta_X) - 1;
        
        R_up = 1 + beta*(1-G_up[i1,i1]);
        R_dn = 1 + beta*(1-G_dn[i1,i1]);
        
        R = R_up*R_dn*exp.(-dT*delta_E_ph);
        
        # phonon field update accepted
        if rand(1)[1] <= abs(R)

            # spin up GF update:
            a_up = (I - G_up);
            b_up = G_up;
            mul!(G_up, -matmul(a_up[:,[i1]],b_up[[i1],:]), (beta/R_up), 1., 1.);
            
            #  spin dn GF update:
            a_dn = (I - G_dn);
            b_dn = G_dn;
            mul!(G_dn, -matmul(a_dn[:,[i1]],b_dn[[i1],:]), (beta/R_dn), 1., 1.);
            
            X[1,i1] = X[1,i1] + delta_X;
            
            # update x of accumulated sign of det
            sgn_accumulated_tmp = sgn_accumulated_tmp * sign(R);
            
            sgn_accumulated_tmp::Float64;
            G_up::Matrix{Float64};
            G_dn::Matrix{Float64};

        end
    end
    h = transpose(h);
    X = transpose(X);
    
    return G_up,G_dn,h,X,sgn_accumulated_tmp
    
end

Sher_Mor (generic function with 1 method)

In [22]:
function Initial_Evaluation(B_up_hat,B_dn_hat,k_vec,N_T)
    
    #Here we use startification & QR decomposition to compute the GF from scratch
    ###################
    #  Stratification
    ###################
    
    # Spin up:
    G_p,log_det_G_p,sign_det_G_p = Strat(B_up_hat,k_vec,N_T);
    
    # Spin down:
    G_m,log_det_G_m,sign_det_G_m = Strat(B_dn_hat,k_vec,N_T);
    
    log_det_G = log_det_G_p + log_det_G_m; #log of abs val of det(G_up*G_dn)
    sign_det_G = sign_det_G_p*sign_det_G_m; #sin of abs val of det(G_dn*G_dn)
    
    sign_det_G::Float64;
    G_p::Matrix{Float64};
    G_m::Matrix{Float64};
    return G_p,G_m,log_det_G,sign_det_G
    
end

Initial_Evaluation (generic function with 1 method)

In [23]:
function Hopping_matrix(t_x,t_y,t_xy,N_x,N_y,bnd_x,bnd_y)
    
    t_x = 1; # hopping to nearest neighbors along x axis
    t_y = t_x; # hopping to nearest neighbors along y axis
    t_xy = 0; ## hopping to next nearest neighbors
    
    # Constructing the matrix of hopping amplitudes for a 2D square lattice
    dx = 1;
    rx_max = dx; #range of hopping along x
    
    if N_x == 1
        rx_max = 0;
    elseif N_x < 5
        rx_max = 1;
    end
    
    dy = 1;
    ry_max = dx; #range of hopping along y
    
    if N_y == 1
        ry_max = 0;
    elseif N_y < 5
        ry_max = 1;
    end
    
    # hopping amplitudes from sit i to its nearest and next nearest neighbors
    T = -[t_xy     t_x   t_xy;
     t_y      0     t_y;
     t_xy     t_x   t_xy];
    
    # hopping matrix
    H0 = zeros(N_x*N_y,N_x*N_y);
    for i1 = 1:N_x, i2 = 1:N_y
        
        r1 = (i1-1)*N_y + i2; #index of site r1 = (i1,i2)
        for rx = -rx_max:rx_max
            j1 = i1 + rx;
            if bnd_x == 1
                j1 = 1 + mod(j1-1,N_x);
            end
            for ry = -ry_max:ry_max
                j2 = i2 + ry;
                if bnd_y == 1
                    j2 = 1 + mod(j2-1,N_y);
                end
                
                r2 = (j1-1)*N_y + j2; #index of site r2 = (j1,j2)
                
                if j1>0 && j1 <=N_x
                    if j2 >0 && j2 <=N_y
                        H0[r1,r2] = T[rx+dx+1,ry+dy+1];
                        H0[r2,r1] = H0[r1,r2]';
                    end
                end
            end
        end
    end
    H0 = (H0 + H0')/2;
    
    #### Translation operators: Tr_px(i) = j where j is the index of i + \hat{x}
    #### Tr_mx(i) = j where j is the index of i - \hat{x}
    #### Tr_py(i) = j where j is the index of i + \hat{y}
    #### Tr_my(i) = j where j is the index of i - \hat{y}

    #### In other words:
    #### Tr_px --> translate along +x by 1 unit cell
    #### Tr_mx --> translate along +x by -1 unit cell
    #### Tr_py --> translate along +y by 1 unit cell
    #### Tr_my --> translate along +y by -1 unit cell
    
    Tr_px = zeros(1,N_x*N_y);
    Tr_py = zeros(1,N_x*N_y);
    
    for i1 = 1 : N_x
        i1_px = mod(i1,N_x) + 1;
        for i2 = 1 : N_y
            i2_py = mod(i2,N_y) + 1;
            r1 = (i1-1)*N_y + i2;
            Tr_px[1,r1] = (i1_px-1)*N_y + i2;
            Tr_py[1,r1] = (i1-1)*N_y + i2_py;
        end
    end
    
    Tr_mx = zeros(1,N_x*N_y);
    Tr_my = zeros(1,N_x*N_y);
    
    for i1 = 1 : N_x
        i1_mx = mod(i1-2,N_x) + 1;
        for i2 = 1 : N_y
            i2_my = mod(i2-2,N_y) + 1;
            r1 = (i1-1)*N_y + i2;
            Tr_mx[1,r1] = (i1_mx-1)*N_y + i2;
            Tr_my[1,r1] = (i1-1)*N_y + i2_my;
        end
    end
    H0::Matrix{Float64}
    return H0,Tr_px,Tr_py,Tr_mx,Tr_my
    
end

Hopping_matrix (generic function with 1 method)

In [24]:
function expectation_values_computation(par_func_tmp,correlations_tmp,n_markov,U,mu,N_x,N_y,t_x,t_y,t_xy,bnd_x,bnd_y,swtch_translation_symmetry,swtch_reflection_symmetry,Tr_px)
    
    N_s = N_x*N_y; #number of sites
    hopping_matrix,Tr_px,Tr_py,Tr_mx,Tr_my = Hopping_matrix(t_x,t_y,t_xy,N_x,N_y,bnd_x,bnd_y); 
    hopping_matrix = hopping_matrix - mu*(Diagonal(ones(N_x*N_y))); #hopping matrix
    
    # combining results of all markov chains
    
    GF_up_avg = correlations_tmp[1];
    GF_dn_avg = correlations_tmp[2];
    n_up_dn_avg = correlations_tmp[3];
    Sgn_avg = par_func_tmp[2];
    
    energy_avg = zeros(n_markov,1);
    for count = 1:n_markov
        ## Average ground-state E
        energy_avg[count,1] = (U*sum(n_up_dn_avg[count]) + tr(hopping_matrix*(GF_up_avg[count] + GF_dn_avg[count])))/N_s;    
    end

    energy_mean = 0;
    Sgn_mean = 0;
    GF_up_mean = zeros(N_s,N_s);
    GF_dn_mean = zeros(N_s,N_s);
    n_up_dn_mean = zeros(N_s);
    
  
    for count = 1:n_markov
        energy_mean = energy_mean + energy_avg[count,1];
        Sgn_mean = Sgn_mean + abs(Sgn_avg[count][1]);
        GF_up_mean = GF_up_mean + GF_up_avg[count];
        GF_dn_mean = GF_dn_mean + GF_dn_avg[count];
        n_up_dn_mean = n_up_dn_mean + n_up_dn_avg[count];
        
    end
    
    
    energy_mean = energy_mean/n_markov;
    Sgn_mean = Sgn_mean/n_markov;
    GF_up_mean = GF_up_mean/n_markov;
    GF_dn_mean = GF_dn_mean/n_markov;
    
    if swtch_translation_symmetry == 1
        v0 = collect(1:N_x*N_y);
        A_up = zeros(N_s,N_s);
        A_dn = zeros(N_s,N_s);
        B = zeros(N_s);
        for i1 = 1:N_x
            A_up = A_up + GF_up_mean[v0,v0];
            A_dn = A_dn + GF_dn_mean[v0,v0];
            B = B + n_up_dn_mean[v0];
            v0 = Int.(Tr_px[v0]);
        end
        GF_up_mean = A_up/N_x;
        GF_up_mean = (GF_up_mean + GF_up_mean')/2;
        GF_dn_mean = A_dn/N_x;
        GF_dn_mean = (GF_dn_mean + GF_dn_mean')/2;
        n_up_dn_mean = B/N_x;
    end
    
    if swtch_reflection_symmetry == 1
        v0 = collect(1:N_x*N_y);
        v0 = reshape(v0,(N_y,:));
        v0 = reverse(v0, dims = 1);
        v0 = reshape(v0,(1,:));
        v0 = vec(v0);
        GF_up_mean = (GF_up_mean + GF_up_mean[v0,v0])/2;
        GF_dn_mean = (GF_dn_mean + GF_dn_mean[v0,v0])/2;
        n_up_dn_mean = (n_up_dn_mean + n_up_dn_mean[v0])/2;
    end

    n_up_mean = diag(GF_up_mean);
    n_dn_mean = diag(GF_dn_mean);
    n_up_dn_mean = n_up_dn_mean/n_markov;

    
    correlations = Dict();
    correlations[1] = Sgn_avg;
    GF_mean = [GF_up_mean,GF_dn_mean];
    correlations[2] = GF_mean;
    correlations[3] = n_up_dn_mean;
    correlations[4] = energy_avg;

    # normalized onsite correlation
    mean_onsite_corr = mean(n_up_dn_mean)/(mean(n_up_mean)*mean(n_dn_mean));
    
    # average density
    n_mean = mean(n_up_mean) + mean(n_dn_mean);
    
    # average kinetic energy
    kinetic_energy_mean = energy_mean -U*mean(n_up_dn_mean);
    
    # average Hubbard interaction energy
    interaction_energy_mean = U*mean(n_up_dn_mean);
    
    # normalized statistical error bar of DQMC calculations
    err_bar = 10^2*std(energy_avg)/abs(mean(energy_avg));
    
    
    measurements = [kinetic_energy_mean interaction_energy_mean n_mean mean_onsite_corr energy_mean err_bar Sgn_mean]
    @printf("measurements:\nkinetic_energy: %s\t interaction_energy: %s\t density: %s\n normalized_onsite_correlation:  %s\tenergy:  %s\n statistical_error_bar: %s\t average_sign: %s\n",kinetic_energy_mean,interaction_energy_mean,n_mean,mean_onsite_corr,energy_mean,err_bar,Sgn_mean)
    return correlations,measurements,n_up_dn_mean,energy_avg
end

expectation_values_computation (generic function with 1 method)

In [25]:
U = 4.0; # Hubbardd interaction strength
mu = 0.0; # chemical ponetial (half-filling --> mu =0), hole doping --> mu <0,  electron doping --> mu >0
g = 0.5;  # elecrtron - phonon coupling constant (lambda)

N_x = 4;   # number of sites along x direction
N_y = 4;  # number of sites along y direction
bnd_x = 1; # boundary condition along x:  0 --> open  1--> periodic
bnd_y = 1; # boundary condition along y:  0 --> open  1--> periodic

beta = 4; # inverse temperature:    beta = 1/T
dT = 1/10; # time steps
xscale = sqrt(U^3)/g^2 * dT
dk = 10; # stratificatoon period (default = 10)
ds = 1;
N_sw_measurement = 10000; ### number of measurment sweeps
N_sw_warm_up = max(1,round(N_sw_measurement/5)) + round(N_sw_measurement/10); ### number of warmup (burn-in) sweeps
n_markov = 1; ### number of independent markov chains
measurement_time_step = 1; ## 1--> measures every time step  &  2--> measures every dk time steps &      3--> measures after one sweep

swtch_correlation = 1; ## determines whether or not the computation of correlation functions are desired

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#           Hopping amplitudes
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
t_x = 1; # hopping to nearest neighbors along x axis
t_y = t_x; # hopping to nearest neighbors along y axis
t_xy = 0; ## hopping to next nearest neighbors
hopping_matrix,Tr_px,Tr_py,Tr_mx,Tr_my = Hopping_matrix(t_x,t_y,t_xy,N_x,N_y,bnd_x,bnd_y);
hopping_matrix = hopping_matrix - mu*(Diagonal(ones(N_x*N_y))); ### considering the chemical potential to enforce the desired total electron average denisty

D0, U0 = eigen(hopping_matrix);
D0 = convert(Vector{Float64},D0);
U0 = convert(Matrix{Float64},U0);
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#           DQMC algorithm
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
M =  1;
Omega = t_x;
W = 8*t_x;
gamma = g^2/(M*Omega^2*W); # g = sqrt(gamma*M*W)*Omega

@time begin
### main compuations are done in func_DQMC function
par_func_tmp,correlations_tmp,abs_log = func_DQMC(hopping_matrix,D0,U0,g,M,Omega,U,beta,N_x,N_y,dT,dk,n_markov,N_sw_warm_up,N_sw_measurement,measurement_time_step,swtch_correlation);
end

#%%%%%%%%%%%%%%%%%%%%%%%%%%
#        Measurements
#%%%%%%%%%%%%%%%%%%%%%%%%%%

# to implement translation symmetry (along x) or reflection symmetry (along
# x) for more accurate computation of quantities/expectation values
swtch_translation_symmetry = 1;
swtch_reflection_symmetry = 1;

### working on outputs of func_DQMC to generate the desired expection values
### in expectation_values_computation function
correlations,measurements,n_up_dn_mean,energy_avg = expectation_values_computation(par_func_tmp,correlations_tmp,n_markov,U,mu,N_x,N_y,t_x,t_y,t_xy,bnd_x,bnd_y,swtch_translation_symmetry,swtch_reflection_symmetry,Tr_px);
#@printf("time: %s",time)

Progress = [1 100]
110.289776 seconds (235.01 M allocations: 183.030 GiB, 11.70% gc time, 1.54% compilation time)
measurements:
kinetic_energy: -1.2698151186942364	 interaction_energy: 0.966259978442516	 density: 1.0674341844351787
 normalized_onsite_correlation:  0.848031098944175	energy:  -0.30355514025172037
 statistical_error_bar: NaN	 average_sign: 0.944045


# U = 4

| $\lambda$ | sign | $<P_{el}>$  | $<K_{el}>$ | $<n>$ |
| --- | --- | ---- | --- | --- |
| 0    | 1.0 | 0.510308 | -1.31363 |  0.127577 |
| 0.05 | 0.998735 | 0.515562 | -1.31594 |  0.12889 |
| 0.1  | 0.991085 | 0.52884 | -1.31904 | 0.13221 |
| 0.15 | 0.97508 | 0.561407 | -1.31824 | 0.1403517 |
| 0.2  | 0.95676 | 0.604996 | -1.32027 | 0.1512489 |
| 0.25 | 0.9657 | 0.667896 | -1.31812 | 0.166974 |
| 0.3  | 0.9437 | 0.720099 | -1.31389  | 0.18002477 |
| 0.35 | 0.93906 | 0.783666 | -1.30456 | 0.1959165 |
| 0.4  | 0.939029 | 0.857561 | -1.29091 | 0.21439 |
| 0.45 | 0.946908 | 0.94199 | -1.27558 | 0.23549 |
| 0.5  | 0.961471 | 1.04111 |  -1.21129 | 0.260276 |
| 0.55 | 0.96521 |  1.14687 |-1.21129 | 0.2867166 |
| 0.6  | 0.980519 | 1.25767 | -1.08824 | 0.314416  |
| 0.65 | 0.990934 | 1.38767 | -0.899857 | 0.346916 |
| 0.7  | 0.990496 | 1.48605 | -0.865079 | 0.371513 |
| 0.8  | 0.993346 | 1.58309 | -0.772134 | 0.3957719 |
| 0.9  | 0.993276 | 1.66274 | -0.68824 | 0.413406 |
| 1    | 0.987098 | 1.77844 | -0.639575 | 0.444609 |

# U = 6

| $\lambda$ | sign | $<P_{el}>$  | $<K_{el}>$ | $<n>$ |
| --- | --- | ---- | --- | --- |
| 0    | 1.0 | 0.481198 | -1.11527  |  0.127577 |
| 0.05 | 0.998735 | 0.48235 | -1.1173 |  0.12889 |
| 0.1  | 0.991085 | 0.501415 | -1.11954 | 0.13221 |
| 0.15 | 0.97508 | 0.511413 | -1.12236 | 0.1403517 |
| 0.2  | 0.95676 | 0.512571 | -1.12034 | 0.1512489 |
| 0.25 | 0.9657 | 0.530084 | -1.12794 | 0.166974 |
| 0.3  | 0.9437 | 0.55072 | -1.13347  | 0.18002477 |
| 0.35 | 0.93906 | 0.588643 | -1.1396 | 0.1959165 |
| 0.4  | 0.939029 | 0.63304 | -1.14548  | 0.21439 |
| 0.45 | 0.946908 | 0.685297 | -1.15081 | 0.23549 |
| 0.5  | 0.961471 | 0.769387 |  -1.15751 | 0.260276 |
| 0.55 | 0.96521 |  0.87107 |-1.15739 | 0.2867166 |
| 0.6  | 0.980519 | 0.938192 | -1.15709 | 0.314416  |
| 0.65 | 0.990934 | 1.04324 | -1.15883 | 0.346916 |
| 0.7  | 0.990496 | 1.20726 | -1.1542 | 0.371513 |
| 0.75  | 0.993346 | 1.34079 | -1.12989 | 0.3957719 |
| 0.8  | 0.993276 | 1.5214 | -1.10925 | 0.413406 |
| 0.85   | 0.987098 | 1.89007 | -0.972687 | 0.444609 |
| 0.9  | 0.993346 | 2.3233 | -0.772134 | 0.3957719 |
| 0.95  | 0.993276 | 2.41893 | -0.688777 | 0.413406 |
| 1    | 0.987098 | 2.55665 | -0.651059 | 0.444609 |

## U = 8

| $\lambda$ | g | sign | $<P_{el}>$  | $<K_{el}>$ |
| --- | --- | --- | ---- | --- |
| 0 | 0 | 1.0 | 0.505791 | -1.31357 |
| 0.05 | 0.632455532 |0.94043 | 0.939223 | -1.28545 |
| 0.1 | 0.894427191 | 0.969925 | 1.32147 | -1.15472 |
| 0.15 | 1.095445115 | 0.97762 | 1.57351 | -1.08615 |
| 0.2 | 1.264911064 | 0.983115 | 1.69873 | -0.971703 |
| 0.25 | 1.414213562 | 0.990205 | 2.08502 | -0.841556 |
| 0.3 | 1.549193338 | 0.991955 | 2.23068 | -0.745794  |
| 0.35 | 1.673320053 | 0.992595 | 2.29958 |-0.677064 |
| 0.4 | 1.788854382 | 0.993435 | 2.50117 | -0.606209 |
| 0.45 | 1.897366596 | 0.99229 | 2.5128 | -0.54872 |
| 0.5 | 2 | 0.99229 | 2.70435 |-0.471361 |
| 0.55 | 2.097617696 | 0.998 |  2.9378 |-0.399385|
| 0.6 | 2.19089023 | 0.99665 | 3.10321 | -0.325361 |
| 0.65 | 2.28035085 | 0.99848 | 3.14819 |-0.310543 |
| 0.7 | 2.366431913 | 0.99678 | 3.24573 | -0.257048 |
| 0.75 | 2.449489743 | 0.99185 | 3.25217 |-0.226345 |